In [1]:
import csv
from collections import defaultdict

data = []
for t in ['train', 'dev', 'test']:
  with open('MAUD_{}.csv'.format(t)) as f:
    for row in csv.DictReader(f):
      row['split'] = t
      data.append(row)

questions = list(set([r['category'] + '|' + r['question'] for r in data]))
questions.sort()

rows = []
count = defaultdict(lambda _=0: defaultdict(int))
with open('MAUD_summary_abridged.csv', 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['category', 'question', 'count', 'avg_num_words', 'max_num_words'])
  for question in questions:
    category, question = question.split('|')
    match = [x for x in data if x['question'] == question and x['category'] == category and \
             x['data_type'] == 'abridged']
    avg_num_words = (sum([len(x['text'].split()) for x in match]) / len(match)) if len(match) > 0 else '-'
    max_num_words = max([len(x['text'].split()) for x in match]) if len(match) > 0 else '-'
    rows.append([category, question, len(match), avg_num_words, max_num_words])
    count[category][question] = len(match)
  rows.sort(key=lambda x: -x[2])
  writer.writerows(rows)

In [2]:
answers = defaultdict(lambda _=0: defaultdict(list))

with open('MAUD deal point questions and answers.txt') as f:
  for line in f.readlines():
    line = line.strip()
    if line.startswith('## CATEGORY: '):
      category = line[13:]
    elif line.startswith('QUESTION '):
      question = line[9:]
      question = question[question.index(' '):].strip()
    elif line.startswith('ANSWER '):
      answer = line[7:]
      answer = answer[answer.index(' '):].strip()
      answers[category][question].append(answer)

In [3]:
with open('MAUD_questions - question_text.csv') as f:
    reader = csv.DictReader(f)
    questions = [x for x in reader]

In [4]:
import os
import random
from string import ascii_lowercase

for q in questions:
  with open('README.md') as f:
    template = f.read()
  category = q['category']
  question = q['question']
  if count[category][question] < 50:
    continue
  is_multilabel = 'multilabel' in question.lower()
  text = q['question_text']
  if len(text) == 0:
    continue
  small_text = text[0].lower() + text[1:]
  match = [x for x in data if x['question'] == question and x['category'] == category and \
           x['data_type'] == 'abridged']

  random.seed(2023)
  random.shuffle(match)
  example = match[0]
  match = match[1:]
  
  a = answers[category][question]
  if 'answers' in q and len(q['answers'].strip()) > 0:
    a = q['answers'].split('\n')
    a = [x[9:].strip() for x in a]
    a = [x[x.index(' '):].strip() for x in a]
  options = ['{}: {}'.format(ascii_lowercase.upper()[idx], option) for idx, option in enumerate(a)]
  options = '\n'.join(options)

  if 'multilabel' in question.lower():
    question = question[:question.rindex('[')].strip()
  if 'Y/' in question:
    question = question[:question.rindex('Y/')].strip()
    if question.endswith('('):
      question = question[:-1].strip()
  if '-answer' in question.lower():
    question = question[:question.lower().rindex('-answer')].strip()
  cleaned_question = question
  while '/' in cleaned_question:
    cleaned_question = cleaned_question.replace('/', '_')
  folder = 'tasks/MAUD_{}'.format(cleaned_question)
  if is_multilabel:
    folder = 'tasks/MAUD_Multi Label_{}'.format(cleaned_question)
  if not os.path.exists(folder):
    os.makedirs(folder)
  title = question
  summary = 'Read the following merger agreement and answer: {}'.format(small_text)
  description = 'the answer that best characterizes'
  if is_multilabel:
    description = 'the answer(s) that correctly characterizes'
  label = 'answer key to the question'
  if is_multilabel:
    label = 'answer key to the question seperated by comma'
  
  with open('{}/README.md'.format(folder), 'w') as f:
    f.write(template.format(title=title,
                            summary=summary,
                            samples=len(match) + 1,
                            description='select {} the merger agreement.'.format(description),
                            question=summary,
                            options=options,
                            answer_type='{} the agreement'.format(description),
                            label=label))
  
  def clean_text(text):
    while '\n' in text:
      text = text.replace('\n', ' ')
    return text
  with open('{}/base_prompt.txt'.format(folder), 'w') as f:
    template = 'base_prompt_multi_label.txt' if is_multilabel else 'base_prompt_single_label.txt'
    with open(template) as ff:
      template = ff.read()
    options = options.split('\n')
    options = ['Option ' + x for x in options]
    options = '\n'.join(options)
    f.write(template.format(question=text,
                            options=options,
                            example_text=clean_text(example['text']),
                            example_answer=ascii_lowercase.upper()[int(example['label'])])
                    .replace('||text||', '{{text}}'))
  
  with open('{}/base_prompt_wo_example.txt'.format(folder), 'w') as f:
    template = 'base_prompt_single_label_wo_example.txt'
    with open(template) as ff:
      template = ff.read()
    f.write(template.format(question=text,
                            options=options)
                    .replace('||text||', '{{text}}'))

  with open('{}/train.tsv'.format(folder), 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
    writer.writerow(['label', 'text'])
    writer.writerow([ascii_lowercase.upper()[int(example['label'])], clean_text(example['text'])])

    
  with open('{}/test.tsv'.format(folder), 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
    writer.writerow(['label', 'text'])
    for m in match:
      writer.writerow([ascii_lowercase.upper()[int(m['label'])], clean_text(m['text'])])


In [17]:
import os
import random
from string import ascii_lowercase

for q in questions:
  with open('README.md') as f:
    template = f.read()
  category = q['category']
  question = q['question']
  if count[category][question] < 50:
    continue
  is_multilabel = 'multilabel' in question.lower()
  text = q['question_text']
  if len(text) == 0:
    continue
  small_text = text[0].lower() + text[1:]
  match = [x for x in data if x['question'] == question and x['category'] == category and \
           x['data_type'] == 'abridged']

  random.seed(2023)
  random.shuffle(match)
  while len(match[0]['text'].split()) > 450:
    random.shuffle(match)
  example = match[0]
  match = match[1:]
  
  a = answers[category][question]
  if 'answers' in q and len(q['answers'].strip()) > 0:
    a = q['answers'].split('\n')
    a = [x[9:].strip() for x in a]
    a = [x[x.index(' '):].strip() for x in a]
  options = ['{}: {}'.format(ascii_lowercase.upper()[idx], option) for idx, option in enumerate(a)]
  options = '\n'.join(options)

  if 'multilabel' in question.lower():
    question = question[:question.rindex('[')].strip()
  if 'Y/' in question:
    question = question[:question.rindex('Y/')].strip()
    if question.endswith('('):
      question = question[:-1].strip()
  if '-answer' in question.lower():
    question = question[:question.lower().rindex('-answer')].strip()
  cleaned_question = question
  while '/' in cleaned_question:
    cleaned_question = cleaned_question.replace('/', '_')
  folder = 'tasks/MAUD_{}'.format(cleaned_question)
  if is_multilabel:
    folder = 'tasks/MAUD_Multi Label_{}'.format(cleaned_question)
  if not os.path.exists(folder):
    os.makedirs(folder)
  title = question
  summary = 'Read the following merger agreement and answer: {}'.format(small_text)
  description = 'the answer that best characterizes'
  if is_multilabel:
    description = 'the answer(s) that correctly characterizes'
  label = 'answer key to the question'
  if is_multilabel:
    label = 'answer key to the question seperated by comma'

  # def clean_text(text):
  #   while '\n' in text:
  #     text = text.replace('\n', ' ')
  #   return text

  print('\n'.join(['\\textbf{Task name}: ' + folder[11:].replace('&', '\\&').replace('%', '\\%') + ' \\\\',
                   '\\textbf{Question}: ' + text.replace('&', '\\&').replace('%', '\\%') + ' \\\\',
                   '\\textbf{Options}: ' + '; '.join(options.split('\n')).replace('&', '\\&').replace('%', '\\%') + ' \\\\',
                   '\\textbf{Example}: ' + example['text'].replace('&', '\\&').replace('%', '\\%') + ' \\\\',
                   '\\textbf{Answer}: ' + ascii_lowercase.upper()[int(example['label'])].replace('&', '\\&').replace('%', '\\%') + ' \\\\']))
  print('\\midrule')
  
  # with open('{}/README.md'.format(folder), 'w') as f:
  #   f.write(template.format(title=title,
  #                           summary=summary,
  #                           samples=len(match) + 1,
  #                           description='select {} the merger agreement.'.format(description),
  #                           question=summary,
  #                           options=options,
  #                           answer_type='{} the agreement'.format(description),
  #                           label=label))
  
  # def clean_text(text):
  #   while '\n' in text:
  #     text = text.replace('\n', ' ')
  #   return text
  # with open('{}/base_prompt.txt'.format(folder), 'w') as f:
  #   template = 'base_prompt_multi_label.txt' if is_multilabel else 'base_prompt_single_label.txt'
  #   with open(template) as ff:
  #     template = ff.read()
  #   options = options.split('\n')
  #   options = ['Option ' + x for x in options]
  #   options = '\n'.join(options)
  #   f.write(template.format(question=text,
  #                           options=options,
  #                           example_text=clean_text(example['text']),
  #                           example_answer=ascii_lowercase.upper()[int(example['label'])])
  #                   .replace('||text||', '{{text}}'))
  

\textbf{Task name}: Type of Consideration \\
\textbf{Question}: What type of consideration is specified in this agreement? \\
\textbf{Options}: A: All Cash; B: All Stock; C: Mixed Cash/Stock; D: Mixed Cash/Stock: Election \\
\textbf{Example}: each Share <omitted> shall be converted into the right to receive the Offer Price in cash, without interest (the “Merger Consideration”), minus any withholding of Taxes required by applicable Laws in accordance with Section 3.6(d) (Page 20) \\
\textbf{Answer}: A \\
\midrule
\textbf{Task name}: Accuracy of Target "General" R\&W: Bringdown Timing Answer \\
\textbf{Question}: When are representations and warranties required to be made according to the bring down provision? \\
\textbf{Options}: A: At Closing Only; B: At Signing \& At Closing \\
\textbf{Example}: Section 7.2 Conditions to Obligations of Parent and Acquisition Sub to Effect the Merger. The obligations of Parent and AcquisitionSub to effect the Merger are, in addition to the conditions s

## Interpretation Variation

In [26]:
import csv
from collections import defaultdict

with open('main-merged.csv') as f:
  data = [r for r in csv.DictReader(f)]

types = {}
questions = defaultdict(lambda _=0: defaultdict(lambda _=0: {
    'yes': 0,
    'no': 0,
    'cantdecide': 0
}))
for r in data:
    questions[r['header']][r['continuation']][r['individual_judgment']] += 1

In [54]:
import json
import random
from statsmodels.stats.proportion import multinomial_proportions_confint

dataset = []
with open('interpretation-variation.csv', 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['policy', 'claim', 'y', 'n', 'cd', 'CI_y', 'CI_n'])
  for q, continuations in questions.items():
    for c, responses in continuations.items():
      responses = [responses['yes'], responses['cantdecide'], responses['no']]
      intervals = multinomial_proportions_confint(responses, alpha=.05)
      intervals = [intervals[0][0], intervals[2][0]]
      writer.writerow([q, c, responses[0], responses[2], responses[1], intervals[0], intervals[1]])

      answer = 'C'
      if intervals[0] >= .5:
        answer = 'A'
      elif intervals[1] >= .5:
        answer = 'B'
      dataset.append({
        'policy': q.replace('<b>', '').replace('</b>', '').strip(),
        'claim': c.strip(),
        'answer': answer
      })

random.seed(2023)
random.shuffle(dataset)
while len(set([x['answer'] for x in dataset[:5]])) != 3:
  random.shuffle(dataset)
trains = dataset[:5]
tests = dataset[5:]

with open('tasks/insuance_policy_interpretation/train.csv', 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['policy', 'claim', 'answer'])
  for t in trains:
    writer.writerow([t['policy'], t['claim'], t['answer']])

with open('tasks/insuance_policy_interpretation/tests.csv', 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['policy', 'claim', 'answer'])
  for t in tests:
    writer.writerow([t['policy'], t['claim'], t['answer']])

template = open('tasks/insuance_policy_interpretation/base_prompt_wo_example.txt').readlines()

for row in trains:
  template = template[:2] + [
    'Policy: {}\n'.format(row['policy']),
    'Claim: {}\n'.format(row['claim']),
    'Label: {}\n'.format(row['answer']),
    '\n'
  ] + template[2:]
with open('tasks/insuance_policy_interpretation/base_prompt.txt', 'w') as f:
  f.writelines(template)

In [33]:
len(dataset)

138

In [49]:
multinomial_proportions_confint([1, 1, 19], alpha=.05)

array([[0.00751476, 0.24822126],
       [0.00751476, 0.24822126],
       [0.68998817, 0.97593223]])

In [53]:
from collections import Counter
Counter([x['answer'] for x in dataset])

Counter({'B': 30, 'A': 48, 'C': 60})